# Introduction

In this notebook, I will get current ratings for 50,000 Lichess players and save them to my local player-opening database.

## The Problem

Ope! I did a silly. I forgot to get the ratings of all 50,000 lichess players in my database.

## The Solution

Luckily, this is an easy problem to solve. Here are the steps:

### 1. Database

- Update the database schema in db_utils to take a `rating` integer on the player table
- Note that this database has already been made so we're not actually changing anything immediately by doing this.  But if the schema is ever used to duplicate or remake the DB, we'll want that rating in there.

### 2. Update existing DB
- In step 1, we updated our schema but that'll only add `rating` if the schema is run again to make a new DB
- So, in this notebook, we'll add a simple integer `rating` column to `player`

### 3. Get player ratings

1. Get all 50,000 player usernames from our DB
2. In batches of 300 usernames, call POST lichess.org/api/users with my lichess auth token (LICHESS_TOKEN), with the usernames in an array
3. For each batch, do a bulk update of usernames in the DB.

NOTE ON API LIMITS:
- We can only get max 8,000 users every ten minutes, and max 120,000 users per day
- The 120k limit shouldn't be an issue since we only have 50k players
- But we'll need simple timeouts and backoffs. Two requests per minute (of 300 players each) seems sensible; that's 6,000 players per ten minutes, well within the limit.
- So this will take about 90 minutes to run, which is fine.

## Rating specs:

We use blitz, rapid and classical games for our data. So there's some question about which rating to use. We'll employ the following formula and specs:

- Blitz is the most common time control in our dataset, so we'll prefer blitz.
- If rating deviation (RD) is <110, use blitz rating
- If blitz RD is >110 and Rapid RD more than 20 points below Blitz RD, use Rapid rating
- Only use Classical rating if both Blitz and Rapid are more than 110, and Classical is less than 110
- If all else fails, use Blitz rating
- The player will always have at least one of these ratings because we're only using highly active players, but if somehow they don't, use NAN or something else that makes sense